### Pre-Processing Data

In [6]:
import pandas as pd
import numpy as np
import ast

In [7]:
sample_sub = pd.read_csv('sample_submission.csv')
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

train.head()


,id,context,question,answers,label
0,train_0,"In rheumatoid arthritis, the body' s immune sy...","The statements above, if true, most strongly s...",['Unlike aspirin and other medications that re...,1
1,train_1,Patient: Pharmacists maintain that doctors sho...,The patient's argument proceeds by,['attempting to discredit a position by questi...,0
2,train_2,Paula will visit the dentist tomorrow morning ...,The pattern of reasoning displayed above most ...,"['If Marge goes to the bank today, Lauren will...",1
3,train_3,Some theorists argue that literary critics sho...,The argument's conclusion follows logically if...,"[""Any critic who is able to help readers make ...",1
4,train_4,Shipping Clerk: The five specially ordered shi...,"If the shipping clerk's statements are true, w...",['At least one of the shipments sent to Truax ...,0


In [8]:
# Splitting up the answers into a list of answers
train['answers'] = train['answers'].apply(ast.literal_eval)

# Creating a column for the correct answer based on the label
train['correct_answer'] = train.apply(lambda row: row['answers'][row['label']], axis=1)


In [9]:
# from datasets import load_dataset
# swag = load_dataset("swag", "regular")

In [10]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

In [11]:
train['answer0'] = train['answers'].apply(lambda x: x[0])
train['answer1'] = train['answers'].apply(lambda x: x[1])
train['answer2'] = train['answers'].apply(lambda x: x[2])
train['answer3'] = train['answers'].apply(lambda x: x[3])

In [12]:
import json

# Assuming your DataFrame is named 'df'

# Convert the entire DataFrame to a JSON string
json_str = train.to_json(orient='records')

# Parse the JSON string into a JSON array (list of JSON objects)
json_data = json.loads(json_str)

# Specify the file name for the JSON file
file_name = 'dataset.json'

# Writing the list of JSON objects to a file
with open(file_name, 'w') as file:
    json.dump(json_data, file, indent=4)


In [13]:
from datasets import DatasetDict
ds = DatasetDict.from_json({'train': 'dataset.json'})

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [14]:
train_test_split = ds["train"].train_test_split(test_size=0.3)

# Combine the new train and validation set with the other sets in the original DatasetDict
ds = DatasetDict({
    'train': train_test_split['train'],
    'validation': train_test_split['test'],
})

In [15]:
ds

DatasetDict({
    train: Dataset({
        features: ['label', 'answer0', 'answer1', 'answer2', 'correct_answer', 'question', 'answers', 'answer3', 'context', 'id'],
        num_rows: 3246
    })
    validation: Dataset({
        features: ['label', 'answer0', 'answer1', 'answer2', 'correct_answer', 'question', 'answers', 'answer3', 'context', 'id'],
        num_rows: 1392
    })
})

In [16]:
ending_names = ["answer0", "answer1", "answer2", "answer3"]

def preprocess_function(examples):
    first_sentences = [[context] * 4 for context in examples["context"]]
    question_headers = examples["question"]
    second_sentences = [
        [f"{header} {examples[end][i]}" for end in ending_names] for i, header in enumerate(question_headers)
    ]

    first_sentences = sum(first_sentences, [])
    second_sentences = sum(second_sentences, [])

    tokenized_examples = tokenizer(first_sentences, second_sentences, truncation=True)
    return {k: [v[i : i + 4] for i in range(0, len(v), 4)] for k, v in tokenized_examples.items()}


In [17]:
tokenized_ds = ds.map(preprocess_function, batched=True)


Map:   0%|          | 0/3246 [00:00<?, ? examples/s]

Map:   0%|          | 0/1392 [00:00<?, ? examples/s]

### Evaluation metrics

In [19]:
import evaluate

accuracy = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


### Training Model

In [20]:
from dataclasses import dataclass
from transformers.tokenization_utils_base import PreTrainedTokenizerBase, PaddingStrategy
from typing import Optional, Union
import tensorflow as tf


@dataclass
class DataCollatorForMultipleChoice:
    """
    Data collator that will dynamically pad the inputs for multiple choice received.
    """

    tokenizer: PreTrainedTokenizerBase
    padding: Union[bool, str, PaddingStrategy] = True
    max_length: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None

    def __call__(self, features):
        label_name = "label" if "label" in features[0].keys() else "labels"
        labels = [feature.pop(label_name) for feature in features]
        batch_size = len(features)
        num_choices = len(features[0]["input_ids"])
        flattened_features = [
            [{k: v[i] for k, v in feature.items()} for i in range(num_choices)] for feature in features
        ]
        flattened_features = sum(flattened_features, [])

        batch = self.tokenizer.pad(
            flattened_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="tf",
        )

        batch = {k: tf.reshape(v, (batch_size, num_choices, -1)) for k, v in batch.items()}
        batch["labels"] = tf.convert_to_tensor(labels, dtype=tf.int64)
        return batch

In [21]:
from transformers import AutoModelForMultipleChoice, TrainingArguments, Trainer

model = AutoModelForMultipleChoice.from_pretrained("bert-base-uncased")

Some weights of BertForMultipleChoice were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [22]:
from transformers import create_optimizer

batch_size = 16
num_train_epochs = 2
total_train_steps = (len(tokenized_ds["train"]) // batch_size) * num_train_epochs
optimizer, schedule = create_optimizer(init_lr=5e-5, num_warmup_steps=0, num_train_steps=total_train_steps)

In [23]:
from transformers import TFAutoModelForMultipleChoice

model = TFAutoModelForMultipleChoice.from_pretrained("bert-base-uncased")

All PyTorch model weights were used when initializing TFBertForMultipleChoice.

Some weights or buffers of the TF 2.0 model TFBertForMultipleChoice were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [24]:
data_collator = DataCollatorForMultipleChoice(tokenizer=tokenizer)
tf_train_set = model.prepare_tf_dataset(
    tokenized_ds["train"],
    shuffle=True,
    batch_size=batch_size,
    collate_fn=data_collator,
)

tf_validation_set = model.prepare_tf_dataset(
    tokenized_ds["validation"],
    shuffle=False,
    batch_size=batch_size,
    collate_fn=data_collator,
)

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [25]:
model.compile(optimizer=optimizer)

In [26]:
from transformers.keras_callbacks import KerasMetricCallback

metric_callback = KerasMetricCallback(metric_fn=compute_metrics, eval_dataset=tf_validation_set)

In [27]:
model.fit(x=tf_train_set, validation_data=tf_validation_set, epochs=2, callbacks=metric_callback)


Epoch 1/2


KeyboardInterrupt: 